In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import math
TRAIN_FILE = 'data/traindata.csv'
TEST_FILE = 'data/testdata.csv'
c = 1
k = 2

In [2]:
df = pd.read_csv(TRAIN_FILE,sep=',',header=None)

In [3]:
data = np.array(df)

In [4]:
ind = data[:,0]
comment = data[:,5]

In [5]:
# Leave each word only once per string
unik = lambda s: ' '.join(set(s.split()))
vunik = np.vectorize(unik)
comment = vunik(comment)

In [51]:
m = len(comment)
print(m)

1600000


In [52]:
phiY = np.sum(ind)/(4.0*m)  ## y==4
num0 = (1-phiY)*m
num1 = m - num0

In [53]:
dct0 = Counter()
dct4 = Counter()
str0 = ""
str4 = ""
for i in range(0,len(comment)):
    if(ind[i] == 0):
        str0 += " " + comment[i]
    else:
        str4 += " " + comment[i]
dct0 = Counter(str0.split())
dct4 = Counter(str4.split())

KeyboardInterrupt: 

In [15]:
# log probab with laplacian smoothing 
def p(x,y):
    if(y==0):
        return math.log((float)(dct0[x]+c)/(float)(num0+(c*k)));
    else:
        return math.log((float)(dct4[x]+c)/(float)(num1+(c*k)));

In [16]:
def py4x(x):
    s = set(x.split())
    p0=0
    p4=0
    for val in s:
        p0 += p(val,0)
        p4 += p(val,4)
    p0 = math.exp(p0)*(1-phiY)
    p4 = math.exp(p4)*(phiY)
    return p4/(p0+p4);

In [17]:
#testing
tdf = pd.read_csv(TEST_FILE,sep=',',header=None)
tdata = np.array(tdf)
tind = tdata[:,0]
tcomment = tdata[:,5]

In [18]:
error = np.zeros(2)
total = 0
cnfmat = np.zeros((2,2)) 
for i in range(0,len(tcomment)):
    if(tind[i] == 2):
        continue;
    total+=1;
    p1 = py4x(tcomment[i])
    clss = 0
    if(p1 > 0.5):
        clss = 4;
    if(clss == tind[i]):
        error[0]+=1;
    else:
        error[1]+=1;
    ## confusion matrix
    cnfmat[clss/4][tind[i]/4]+=1

In [19]:
print(error/total)

[0.80501393 0.19498607]


In [20]:
cnfmat

array([[154.,  47.],
       [ 23., 135.]])

In [48]:
# part(b) Baseline accuracy
# random guessing
import random
total = 0
error = np.zeros(2)
for i in range(0,len(tcomment)):
    if(tind[i]==2):
        continue;
    total+=1;
    p1 = random.random()
    clss = 0
    if(p1 > 0.5):
        clss = 4;
    if(clss == tind[i]):
        error[0]+=1;
    else:
        error[1]+=1;
print(error/total)
# Theoretically must be 0.5 with very large data set

total = 0
error = np.zeros(2)
for i in range(0,len(tcomment)):
    if(tind[i]==2):
        continue;
    total+=1;
    p1 = phiY
    clss = 0
    if(p1 > 0.5):
        clss = 4;
    if(clss == tind[i]):
        error[0]+=1;
    else:
        error[1]+=1;
print(error/total)
#equal to the num of majority class present.

[0.48746518 0.51253482]
[0.49303621 0.50696379]


In [5]:
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import string
import unicodedata
tknzr = TweetTokenizer(strip_handles=True)
ps = nltk.stem.PorterStemmer()

In [45]:
#stemming
def stem(s):
    # remove twitter handle then digits and punctuation
    s = s.decode('latin-1')
    words = set([ps.stem(word.lower()) for word in tknzr.tokenize(s) if word.isalpha() and word not in stopwords.words('english') ]) 
    s = ' '.join(words)
    if type(s) is not unicode:
        return s.decode('latin-1')
    return s

In [47]:
scomment = []
for i in range(0,len(comment)):
    if(i%1000==0):
        print(i)
#     print(i)
    scomment.append(stem(comment[i]))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


1164000
1165000
1166000
1167000
1168000
1169000
1170000
1171000
1172000
1173000
1174000
1175000
1176000
1177000
1178000
1179000
1180000
1181000
1182000
1183000
1184000
1185000
1186000
1187000
1188000
1189000
1190000
1191000
1192000
1193000
1194000
1195000
1196000
1197000
1198000
1199000
1200000
1201000
1202000
1203000
1204000
1205000
1206000
1207000
1208000
1209000
1210000
1211000
1212000
1213000
1214000
1215000
1216000
1217000
1218000
1219000
1220000
1221000
1222000
1223000
1224000
1225000
1226000
1227000
1228000
1229000
1230000
1231000
1232000
1233000
1234000
1235000
1236000
1237000
1238000
1239000
1240000
1241000
1242000
1243000
1244000
1245000
1246000
1247000
1248000
1249000
1250000
1251000
1252000
1253000
1254000
1255000
1256000
1257000
1258000
1259000
1260000
1261000
1262000
1263000
1264000
1265000
1266000
1267000
1268000
1269000
1270000
1271000
1272000
1273000
1274000
1275000
1276000
1277000
1278000
1279000
1280000
1281000
1282000
1283000
1284000
1285000
1286000
1287000
1288000


In [120]:
words =tknzr.tokenize("oh yeah some tweet .... !!! - 123 12 2.34 @saksa")
words=[word.lower() for word in words if word.isalpha()]
print(words)

[u'oh', u'yeah', u'some', u'tweet']


In [49]:
comment = scomment

In [56]:
comment

[u'third carr david awww bummer got you shoulda day',
 u'school blah text upset also facebook result cri might today updat',
 u'ball i dive bound rest time go mani the save manag',
 u'like feel itchi bodi fire whole',
 u'i see behav mad',
 u'whole crew',
 u'need hug',
 u'lol thank ye hey rain see long time bit fine',
 u'nope',
 u'que muera',
 u'break spring citi snow plain',
 u'i ear',
 u'and loss i watch bear thought embarrass ua',
 u'count i anymor never it either talk idk',
 u'though doucheclown zac realli gun first',
 u'i wish premier watch got miss',
 u'death holli watch scene hurt director cut film wri sever',
 u'tax file',
 u'ive love ahh soundtrack alway see rent want',
 u'drink oh were tabl forgotten dear',
 u'much done day get',
 u'sigh mid one call today time ask meet valley friend',
 u'i cake bake ate',
 u'week go hope',
 u'blagh class tomorrow',
 u'i call hate wake peopl',
 u'me just watch sleep go cri marley',
 u'im sad',
 u'ok get i lol ooooh mad lesli',
 u'depress everi

In [15]:
comment[240454]

'faceyourmanga.com \xe1\x80\x80\xe1\x80\xb1\xe1\x80\x80\xe1\x80\xac\xe1\x80\x84\xe1\x80\xb9\xe1\x80\xb8\xe1\x80\x98\xe1\x80\xb0\xe1\x80\xb8 \xe1\x80\x94\xe1\x80\xac\xe1\x80\xb7\xe1\x80\x95\xe1\x80\xb6\xe1\x80\xaf\xe1\x80\x80\xe1\x80\xad\xe1\x80\xaf \xe1\x80\x80\xe1\x80?\xe1\x80\xb6\xe1\x80\xaf\xe1\x80\xb8\xe1\x82\x80\xe1\x80\x80\xe1\x80\xae\xe1\x80\xb8\xe1\x80\x95\xe1\x80\xad\xe1\x80\xaf\xe1\x82\x94\xe1\x80\x9c\xe1\x80\xad\xe1\x80\xaf\xe1\x80\xb9\xe1\x80\x80\xe1\x80\xb9\xe1\x80?\xe1\x80\x9a\xe1\x80\xb9.. '

In [44]:
# s = u'sak\xe1\x80\x80'
# s = comment[240454].encode('ascii', 'ignore')
# print(type(s))
s = comment[240454].decode('latin-1')
# s = 'as as sakahmdhull.com'
print(s)
t = tknzr.tokenize(s)
print(t)
print(set([word for word in t if '\\x'  not in word]))
set([ps.stem(word.lower()) for word in tknzr.tokenize(s) if word.isalpha() and word not in stopwords.words('english') ])
# s = s.translate(None,string.punctuation)

faceyourmanga.com áá±áá¬áá¹á¸áá°á¸ áá¬á·áá¶á¯áá­á¯ áá?á¶á¯á¸ááá®á¸áá­á¯ááá­á¯á¹áá¹á?áá¹.. 
[u'faceyourmanga.com', u'\xe1', u'\x80', u'\x80', u'\xe1', u'\x80', u'\xb1', u'\xe1', u'\x80', u'\x80', u'\xe1', u'\x80', u'\xac', u'\xe1', u'\x80', u'\x84', u'\xe1', u'\x80', u'\xb9\xe1', u'\x80', u'\xb8', u'\xe1', u'\x80', u'\x98', u'\xe1', u'\x80', u'\xb0', u'\xe1', u'\x80', u'\xb8', u'\xe1', u'\x80', u'\x94', u'\xe1', u'\x80', u'\xac', u'\xe1', u'\x80', u'\xb7', u'\xe1', u'\x80', u'\x95', u'\xe1', u'\x80', u'\xb6', u'\xe1', u'\x80', u'\xaf', u'\xe1', u'\x80', u'\x80', u'\xe1', u'\x80', u'\xad', u'\xe1', u'\x80', u'\xaf', u'\xe1', u'\x80', u'\x80', u'\xe1', u'\x80', u'?', u'\xe1', u'\x80', u'\xb6', u'\xe1', u'\x80', u'\xaf', u'\xe1', u'\x80', u'\xb8', u'\xe1', u'\x82', u'\x80', u'\xe1', u'\x80', u'\x80', u'\xe1', u'\x80', u'\xae', u'\xe1', u'\x80', u'\xb8', u'\xe1', u'\x80', u'\x95', u'\xe1', u'\x80', u'\xad', u'\xe1', u'\x80', u'\xaf', u

{u'\xe1'}

In [46]:
stem(comment[240454])

u'\xe1'